In [1]:
import sys
import os
import torch
import cv2
import numpy as np

In [2]:
sys.path.append("../0.mc_utils/")

In [3]:
from openpose.op_utils import play_ob25b_on_video
# from openpose.op_formats import MapOP25bToSMPL24
from common.joints_kalman import KalmanKP2D

from common.tools_cv import draw_kp
from openpose.op_utils import draw_kp2d_to_image,draw_reided_bboxes

In [4]:
video_fname = "E:/20190727-area51/skating_hp_81_02.mp4"

In [5]:
raw_data = np.load("E:/20190727-area51/skating_hp_81_02_op25b_x2ds.npz",allow_pickle=True)
ret_op25b_x2ds = raw_data["op25b"].item()
play_ob25b_on_video(video_fname,ret_op25b_x2ds,interval=100)

In [6]:
class NPMapOP25bToSMPL24:
    """
    convert openpose 25b output format to SMPL24 format.
    based on op25b's confidence
    """
    def __init__(self,threshold=0.2):
        self.select_index = [11,11,12,11,13,14,11,15,16,5,19,22,17,5,6,18,5,6,7,8,9,10,9,10]
        self.mask = np.ones((1,24,1),dtype=np.float32)
        self.mask[:,[3,9,13,14,22,23]] = 0.0
        self.threshold = threshold
        pass
    
    def synthesis_joint(self,masked_op25,viz25,joint_list):
        return np.sum(masked_op25[:,joint_list],axis=1)/(np.sum(viz25[:,joint_list],axis=1)+1e-5)

    def __call__(self,x2ds25b):
        """
        x2ds25b shape is Nx25x(u,v,c)
        """
        assert x2ds25b.shape[1:]==(25,3)
        
        #step1.get availables
        viz25 = (x2ds25b[:,:,-1]>self.threshold).astype(np.int)[...,np.newaxis]#Nx25x1
        masked_op25b = viz25 * x2ds25b
        
        #step2.basic conversion
        x2ds_smpl24 = x2ds25b[:,self.select_index]
        
        #step3.fixing joints
        x2ds_smpl24[:, 0] = self.synthesis_joint(masked_op25b,viz25,[11,12])
        x2ds_smpl24[:,12] = np.where(x2ds_smpl24[:,12,-1:]>self.threshold,
                                        x2ds_smpl24[:,12],
                                        self.synthesis_joint(masked_op25b,viz25,[5,6]))
        x2ds_smpl24[:,15] = np.where(x2ds_smpl24[:,15,-1:]>self.threshold,
                                        x2ds_smpl24[:,15],
                                        self.synthesis_joint(masked_op25b,viz25,[0,1,2,3,4,18]))
        x2ds_smpl24[:, 6] = x2ds_smpl24[:,0]*0.6 + x2ds_smpl24[:,12]*0.4
        
        #step4.final output
        x2ds_smpl24 *= self.mask
        return x2ds_smpl24


In [7]:
class SMPL24Kalman:
    def __init__(self,mask=None,min_frames=20):
        #step1.mask
        if mask is None:
            mask = np.ones(24)
            mask[[3,9,13,14,22,23]] = 0
        
        #step2.create kalmans
        assert mask.shape==(24,)
        self.kalman_dict = {}
        for i in range(24):
            if mask[i]==0:
                continue
            self.kalman_dict[i] = KalmanKP2D()
        
        #step3.empty kalmans
        self.reset()
        self.min_frames = min_frames
        
    def reset(self):
        self.filted_x2ds = np.zeros((24,2),dtype=np.float32)
        self.frame_counter = 0
        
    def x2ds_visible(self,x2ds):
        return (abs(x2ds).sum(-1)>0)
    
    def __call__(self,x2ds):
        """
        x2ds shape is 24x2
        """
        assert x2ds.ndim==2 and x2ds.shape==(24,2)
        viz = self.x2ds_visible(x2ds)        
        for i in range(24):
            #step1.if masked
            if i not in self.kalman_dict:
                self.filted_x2ds[i] = x2ds[i]
                continue
            #step2.if visible
            if viz[i]:
                self.kalman_dict[i].correct(x2ds[i])
            else:
                self.kalman_dict[i].correct(self.filted_x2ds[i])
                
            #step3.predict()
            self.filted_x2ds[i] = self.kalman_dict[i].predict()
        self.frame_counter += 1
        ret_x2ds = self.filted_x2ds if self.frame_counter>self.min_frames else x2ds
        return ret_x2ds

In [8]:
def debug_dump_bboxes(image,kp2ds):
    for i in range(kp2ds.shape[0]):
        draw_reided_bboxes(image,kp2ds[i,...,:2],str(i))
        draw_kp2d_to_image(image,kp2ds[i,...,:2])
    draw_kp(image,kp2ds[...,:2],fmt="smpl24",color=(255,255,255))

In [9]:
def play_smpl24_on_video(video_fname,x2ds_dict,interval=-1):
    #step1.video capture
    cap = cv2.VideoCapture(video_fname)
    fps = cap.get(cv2.CAP_PROP_FPS)
    interval = int(1000//fps) if interval<0 else interval
    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    #step2.map op25b to smpl24
    map_op25b_to_smpl24 = NPMapOP25bToSMPL24()
    smpl24_kalman = SMPL24Kalman()
    
    #step3.start play
    cv2.namedWindow("frame_show")
    for idx in range(frames):
        is_valid,frame = cap.read()
        if not is_valid:
            break
        #draw x2ds to frame
        if idx in x2ds_dict:
            x2ds = map_op25b_to_smpl24(x2ds_dict[idx])
            x2ds = smpl24_kalman(x2ds[0,...,:2])
            debug_dump_bboxes(frame,x2ds[np.newaxis])
            #debug_dump_bboxes(frame,x2ds)
        
        #draw basic info to frame
        cv2.putText(frame, "frame:<{}>".format(idx), (10,25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 1,cv2.LINE_AA)
        cv2.imshow("frame_show",frame)
        if cv2.waitKey(interval) & 0xFF==ord('q'):
            break
    cv2.destroyAllWindows()
    cap.release()

In [10]:
play_smpl24_on_video(video_fname,ret_op25b_x2ds,interval=100)